<a href="https://colab.research.google.com/github/martaarozarena/KSchool-Master-Final-Project/blob/master/exog_varaibles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
from google.colab import drive
#from os.path import join

ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/KSchool-Master-Final-Project'       # path to your project on Drive

drive.mount(ROOT)           # we mount the drive at /content/drive

#PROJECT_PATH = join(ROOT, PROJ)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
cd /content/drive/.shortcut-targets-by-id/1JF1t24jwhTtQKP-lLGdAUHcNKaXnHy3u/KSchool-Master-Final-Project/data

/content/drive/.shortcut-targets-by-id/1JF1t24jwhTtQKP-lLGdAUHcNKaXnHy3u/KSchool-Master-Final-Project/data


In [69]:
import pandas as pd

In [70]:
sheets=["c1_schoolclosing","c2_workplaceclosing","c3_cancelpublicevents","c4_restrictionsongatherings","c5_closepublictransport","c6_stayathomerequirements","c7_domestictravel","c8_internationaltravel"]
url = "https://drive.google.com/file/d/1P37xzRtsjWIMUfhs3uUrHHLyToyE7qOJ/view?usp=sharing"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
varc = pd.read_excel(path,sheet_name=sheets)


In [71]:
j=1
for i in varc.keys():
  varc[i]=varc[i].drop("CountryCode",axis=1)
  varc[i].index=varc[i].CountryName
  varc[i]=varc[i].drop("CountryName",axis=1)
  varc[i]=varc[i][:-3]
  varc[i]=varc[i].T
  varc[i].index=pd.to_datetime(varc[i].index,format="%d%b%Y")
  varc[i]=varc[i].truncate("01-01-2020","30-06-2020")
  #varc[i].to_csv('variable_c{}.csv'.format(j))
  j=j+1

In [72]:
url = "https://drive.google.com/file/d/1P37xzRtsjWIMUfhs3uUrHHLyToyE7qOJ/view?usp=sharing"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
sheets2=["h1_publicinfocampaign","h2_testingpolicy","h3_contacttracing"]
varh = pd.read_excel(path,sheet_name=sheets2)

In [73]:
k=1
for i in varh.keys():
  varh[i]=varh[i].drop("CountryCode",axis=1)
  varh[i].index=varh[i].CountryName
  varh[i]=varh[i].drop("CountryName",axis=1)
  varh[i]=varh[i][:-3]
  varh[i]=varh[i].T
  varh[i].index=pd.to_datetime(varh[i].index,format="%d%b%Y")
  varh[i]=varh[i].truncate("01-01-2020","30-06-2020")
  #varh[i].to_csv('variable_h{}.csv'.format(k))
  k=k+1

In [74]:
varc1=varc['c1_schoolclosing']
varc2=varc["c2_workplaceclosing"]
varc3=varc['c3_cancelpublicevents']
varc4=varc["c4_restrictionsongatherings"]
varc5=varc['c5_closepublictransport']
varc6=varc["c6_stayathomerequirements"]
varc7=varc['c7_domestictravel']
varc8=varc["c8_internationaltravel"]
varh1=varh["h1_publicinfocampaign"]
varh2=varh["h2_testingpolicy"]
varh3=varh["h3_contacttracing"]
varc1.columns="c1_"+varc1.columns
varc2.columns="c2_"+varc2.columns
varc3.columns="c3_"+varc3.columns
varc4.columns="c4_"+varc4.columns
varc5.columns="c5_"+varc5.columns
varc6.columns="c6_"+varc6.columns
varc7.columns="c7_"+varc7.columns
varc8.columns="c8_"+varc8.columns
varh1.columns="h1_"+varh1.columns
varh2.columns="h2_"+varh2.columns
varh3.columns="h3_"+varh3.columns
exogenas=pd.concat([varc1, varc2,varc3,varc4,varc5,varc6,varc7,varc8,varh1,varh2,varh3],axis=1)

In [75]:
#download the yougov data and then choosing the mask wearing data (i12 variable)

In [76]:
url2 = "https://drive.google.com/file/d/1WsBkpNhB5ilLsNxmOZPwdXv66m5tPDHx/view?usp=sharing"
path2 = 'https://drive.google.com/uc?export=download&id='+url2.split('/')[-2]
var_percent_mask = pd.read_csv(path2,sep=";")

In [77]:
var_percent_mask.DateTime=pd.to_datetime(var_percent_mask.DateTime)
var_percent_mask.index=var_percent_mask.DateTime
var_percent_mask.drop("DateTime",axis=1,inplace=True)

In [78]:
var_percent_mask=var_percent_mask.resample("D").mean()

In [79]:
var_percent_mask.interpolate(method='linear',inplace=True)
var_percent_mask.fillna(method="bfill",inplace=True)


In [80]:
var_percent_mask.columns="mask_"+var_percent_mask.columns

In [81]:
var_percent_mask=var_percent_mask.truncate("01-01-2020","30-06-2020")

In [82]:
exogenas=pd.concat([exogenas,var_percent_mask],axis=1)

In [85]:
exogenas.fillna(0,inplace=True)

In [86]:
exogenas.to_csv('exogenas.csv')

In [91]:
exogenas.shape

(182, 1956)